In [1]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

In [2]:
import os
from pathlib import Path

import paramiko
from scp import SCPClient

import getpass
import gc
import time

In [3]:
dir_github = '/media/rich/Home_Linux_partition/github_repos/'
import sys
sys.path.append(dir_github)

%load_ext autoreload
%autoreload 2
from basic_neural_processing_modules import server

----
# ==== Optional ====
### transfer raw data from local machine to `MICROSCOPE`

In [4]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')
pw = server.pw_encode(getpass.getpass(prompt='Password: '))

Username: rh183
Password: ········


In [5]:
sftp = server.sftp_interface(hostname="transfer.rc.hms.harvard.edu", port=22)
sftp.connect(username=username, password=server.pw_decode(pw))

In [6]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    passcode_method=1,
    verbose=1,
    skip_passcode=False,    
)

Duo two-factor login for rh183

Enter a passcode or select one of the following options:

 1. Duo Push to XXX-XXX-4151
 2. Phone call to XXX-XXX-4151
 3. SMS passcodes to XXX-XXX-4151

Passcode or option (1-3): 
1



Pushed a login request to your device...

Success. Logging you in...
Welcome to O2 (Orchestra 2)!

   ___    ____
  / _ \  |___ \
 | | | |   __) |
 | |_| |  / __/
  \___/  |_____|


You've landed on transfer02 which is a
4 core system with 7.62 GiB memory
running kernel 3.10.0 born on 2021-10-22

==== O2 ===================================================================

  News (Jan 3 2022)
  

  +-------------------------------------------------------------------------------+
  | ##################### RC Core launched in July 2021 ######################### |
  |                                                                               |
  | Since July 2021, researchers whose PI does not have a primary or secondary    |
  | faculty appointment in an HMS Basic or Socia

In [7]:
# def mkdir_transfer_check(dir_from, dir_to, sftp, ssh_t, str_expect='[rh183', verbose=1, timeout_cp=10, timeout_diff=10):
#     """
#     Makes a directory
#     Transfers
#     Checks to make sure the transfer worked
#     """
#     ssh_t.receive(timeout=0.1, verbose=False, throw_error=False) ## clean any lingering receives
    
#     sftp.mkdir_p(dir_to)
    
#     if verbose>1:
#         print(f'copy starting,         time: {time.ctime()}, dir:{dir_from}')
#     ssh_t.send(f"cp -r {dir_from}/. {dir_to}/.")
#     test = ssh_t.expect(str_expect, verbose=0, total_timeout=timeout_cp)  ## wait for copy to end
#     if test[0] is None:
#         print('copy expect timed out')
#     elif 'No such file or directory' in test[0]:
#         print(f'copy failed: {test[0]}')
#     elif test[1]==False:
#         if verbose>0:
#             print(f'Post copy expect timed out. time: {time.ctime()}, dir: {dir_from}')
#     else:
#         if verbose>1:
#             print(f'copy expect completed. time: {time.ctime()}, dir: {dir_from}')

#     diff = ssh_t.send(f"diff -qr {dir_from} {dir_to}") ## check differences in directory
#     test = ssh_t.expect('Only in', verbose=0, total_timeout=timeout_diff)
#     if test[0] is None:
#         print('diff expect timed out')
#     elif 'No such file or directory' in test[0]:
#         print(f'diff check failed: {test[0]}')
#     elif verbose>0:
#         if test[1]:
#             print(f"differences found: {test}")
#         else:
#             print(f"diff check passes: No differences found. time: {time.ctime()}, from dir: {dir_from} to dir: {dir_to}")
    
#     ssh_t.receive(timeout=0.1, verbose=False, throw_error=False) ## clean any lingering receives

In [8]:
def transfer(dir_from, dir_to, sftp, ssh_t, str_expect='[rh183', verbose=1, timeout_cp=10):
    """
    Makes a directory
    Transfers
    Checks to make sure the transfer worked
    """
    ssh_t.receive(timeout=0.1, verbose=False, throw_error=False) ## clean any lingering receives
    
    sftp.mkdir_p(dir_to)

    if verbose>1:
        print(f'copy starting,         time: {time.ctime()}, dir:{dir_from}')
    ssh_t.send(f"rsync -av {dir_from}/ {dir_to}/")
    test = ssh_t.expect(str_expect, verbose=1, total_timeout=timeout_cp)  ## wait for copy to end
    if test[0] is None:
        print('copy expect timed out')
    elif 'No such file or directory' in test[0]:
        print(f'copy failed: {test[0]}')
    elif test[1]==False:
        if verbose>0:
            print(f'Post copy expect timed out. time: {time.ctime()}, dir: {dir_from}')
    else:
        if verbose>1:
            print(f'copy expect completed. time: {time.ctime()}, dir: {dir_from}')

    ssh_t.receive(timeout=0.1, verbose=False, throw_error=False) ## clean any lingering receives

In [33]:
## make directory structure on MICROSCOPE

animals = ['AEG21', 'AEG22']
# animals = ['AEG21']
dates = ['2022_05_13', '2022_05_14', '2022_05_16', '2022_05_17', '2022_05_18']
# dates = ['2022_05_13']

In [10]:
# ## make directory structure on MICROSCOPE
# animals = ['AEG21', 'AEG22']
# dates = ['2022_05_13', '2022_05_14', '2022_05_16', '2022_05_17', '2022_05_18']
# methods_MICROSCOPE = ['clean_traces', 'ROI_classification_logisticRegression', 'spike_inference', 'suite2p_output']
# methods_o2 = ['cleanTraces', 'classify_rois', 'spike_inference', 'suite2p_output']

In [50]:
## make directory structure on MICROSCOPE
animals = ['AEG21']
dates = ['2022_05_13', '2022_05_14', '2022_05_16', '2022_05_17', '2022_05_18']
methods_to = ['scanimage']
methods_from = ['Ally/Mesoscope']

In [51]:
## Define directories for where the local data is and the remote directory where the contents of that folder should be copied to (MICROSCOPE)
dir_data_from = '/n/files/Neurobio/MICROSCOPE'
dir_data_to = '/n/files/Neurobio/MICROSCOPE/Rich/data/2pRAM/face_rhythm_motor_mapping'  ## contents of dir_data_local will be dumped into dir_data_MICROSCOPE

[[[transfer(
    dir_from=str(Path(dir_data_from) / method_from / animal / date),
    dir_to=str(Path(dir_data_to) / animal / date / method_to),
    sftp=sftp,
    ssh_t=ssh_t,
    str_expect='[rh183',
    verbose=2,
    timeout_cp=60*60*4,
) for method_from, method_to in zip(methods_from, methods_to)] for date in dates] for animal in animals];

copy starting,         time: Sun Jun  5 12:51:41 2022, dir:/n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_13
tor_mapping/AEG21/2022_05_13/scanimago/MICROSCOPE/Rich/data/2pRAM/face_rhythm_mo 
e/
sending incremental file list
./

AEG21_2022-05-13_0001.h5

Untitled.png

file_00001_00001.tif

file_00001_00002.tif

file_00001_00003.tif

file_00001_00004.tif

file_00001_00005.tif

file_00001_00006.tif

file_00001_00007.tif

file_00001_00008.tif

file_00001_00009.tif

file_00001_00010.tif

file_00001_00011.tif

file_00001_00012.tif

file_00001_00013.tif

file_00001_00014.tif

file_00001_00015.tif

file_00001_00016.tif

file_00001_00017.tif

file_00001_00018.tif

file_00001_00019.tif

file_Motion_00001.csv

other_tifs/

other_tifs/zstack_00001_00001.tif

suite2p_o2_output/

suite2p_o2_output/dispatcher.py
suite2p_o2_output/notes.txt
suite2p_o2_output/remote_run_s2p.py
suite2p_o2_output/jobNum_0/
suite2p_o2_output/jobNum_0/batch_run_output.png

suite2p_o2_output/jobNum_0/log.txt
suit

suite2p/plane0/reg_tif/file131_chan0.tif

suite2p/plane0/reg_tif/file132_chan0.tif

suite2p/plane0/reg_tif/file133_chan0.tif

suite2p/plane0/reg_tif/file134_chan0.tif

suite2p/plane0/reg_tif/file135_chan0.tif

suite2p/plane0/reg_tif/file136_chan0.tif

suite2p/plane0/reg_tif/file137_chan0.tif

suite2p/plane0/reg_tif/file138_chan0.tif

suite2p/plane0/reg_tif/file139_chan0.tif

suite2p/plane0/reg_tif/file140_chan0.tif

suite2p/plane0/reg_tif/file141_chan0.tif

suite2p/plane0/reg_tif/file142_chan0.tif

suite2p/plane0/reg_tif/file143_chan0.tif

suite2p/plane0/reg_tif/file144_chan0.tif

suite2p/plane0/reg_tif/file145_chan0.tif

suite2p/plane0/reg_tif/file146_chan0.tif

suite2p/plane0/reg_tif/file147_chan0.tif

suite2p/plane0/reg_tif/file148_chan0.tif

suite2p/plane0/reg_tif/file149_chan0.tif

suite2p/plane0/reg_tif/file150_chan0.tif

suite2p/plane0/reg_tif/file151_chan0.tif

suite2p/plane0/reg_tif/file152_chan0.tif

suite2p/plane0/reg_tif/file153_chan0.tif

suite2p/plane0/reg_tif/file154_cha

suite2p_o2_output/jobNum_0/suite2p/plane0/Fneu.npy

suite2p_o2_output/jobNum_0/suite2p/plane0/iscell.npy

suite2p_o2_output/jobNum_0/suite2p/plane0/ops.npy

suite2p_o2_output/jobNum_0/suite2p/plane0/spks.npy

suite2p_o2_output/jobNum_0/suite2p/plane0/stat.npy

zstack/

zstack/zstack_00002_00001.tif


sent 90,551,192,483 bytes  received 1,575 bytes  114,694,355.99 bytes/sec
total size is 90,529,085,286  speedup is 1.00
(base) [rh183@transfer04 ~]$ 
copy expect completed. time: Sun Jun  5 13:59:16 2022, dir: /n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_17
copy starting,         time: Sun Jun  5 13:59:16 2022, dir:/n/files/Neurobio/MICROSCOPE/Ally/Mesoscope/AEG21/2022_05_18
tor_mapping/AEG21/2022_05_18/scanimage/MICROSCOPE/Rich/data/2pRAM/face_rhythm_mo 

sending incremental file list

./
AEG21_2022-05-18_0001.h5

EndImaging.png

file_00001_00001.tif

file_00001_00002.tif

file_00001_00003.tif

file_00001_00004.tif

file_00001_00005.tif

file_00001_00006.tif

file_00001_00007

suite2p/plane0/reg_tif/file140_chan0.tif

suite2p/plane0/reg_tif/file141_chan0.tif

suite2p/plane0/reg_tif/file142_chan0.tif

suite2p/plane0/reg_tif/file143_chan0.tif

suite2p/plane0/reg_tif/file144_chan0.tif

suite2p/plane0/reg_tif/file145_chan0.tif

suite2p/plane0/reg_tif/file146_chan0.tif

suite2p/plane0/reg_tif/file147_chan0.tif

suite2p/plane0/reg_tif/file148_chan0.tif

suite2p/plane0/reg_tif/file149_chan0.tif

suite2p/plane0/reg_tif/file150_chan0.tif

suite2p/plane0/reg_tif/file151_chan0.tif

suite2p/plane0/reg_tif/file152_chan0.tif

suite2p/plane0/reg_tif/file153_chan0.tif

suite2p/plane0/reg_tif/file154_chan0.tif

suite2p/plane0/reg_tif/file155_chan0.tif

suite2p/plane0/reg_tif/file156_chan0.tif

suite2p/plane0/reg_tif/file157_chan0.tif

suite2p/plane0/reg_tif/file158_chan0.tif

suite2p/plane0/reg_tif/file159_chan0.tif

suite2p/plane0/reg_tif/file160_chan0.tif

suite2p/plane0/reg_tif/file161_chan0.tif

suite2p/plane0/reg_tif/file162_chan0.tif

suite2p/plane0/reg_tif/file163_cha

In [ ]:
/n/files/Neurobio/MICROSCOPE/Rich/data/2pRAM/face_rhythm_motor_mapping

In [34]:
## Define directories for where the local data is and the remote directory where the contents of that folder should be copied to (MICROSCOPE)
dir_data_from = '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/'
dir_data_to = '/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM'  ## contents of dir_data_local will be dumped into dir_data_MICROSCOPE

[[transfer(
    dir_from=str(Path(dir_data_from) / animal / date),
    dir_to=str(Path(dir_data_to) / animal / date / 'faceRhythm'),
    ssh_t=ssh_t,
    str_expect='[rh183',
    verbose=2,
    timeout_cp=60*60*4,
) for date in dates] for animal in animals];

# [[print(
#     str(Path(dir_data_from) / animal / date),
#     str(Path(dir_data_to) / animal / date / 'faceRhythm'),
#     ) for date in dates] for animal in animals];

copy starting,         time: Sat Jun  4 22:02:39 2022, dir:/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/AEG21/2022_05_13
/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_13/faceRhythm/PE/Rich/analysis_data 

sending incremental file list
created directory /n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_13/faceRhythm
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_13/faceRhythm/." failed: Operation not permitted (1)

rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_13/faceRhythm/jobNum_0" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_13/faceRhythm/jobNum_0/batchRun" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_13/faceRhythm/jobNum_0/b

rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_14/faceRhythm/jobNum_0/.print_log_54350194.log.8een70" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_14/faceRhythm/jobNum_0/.sbatch_config.sh.na3isP" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_14/faceRhythm/jobNum_0/batchRun/configs/.config__batch.yaml.bdPqOD" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_14/faceRhythm/jobNum_0/batchRun/data/.session_batch.nwb.b3B1bs" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_14/faceRhythm/jobNum_0/batchRun/viz/.optic_test.avi.mhE1G9" failed: Operation not permitte

FRbatch.py
dispatcher.py
notes.txt
parameters_batch.json
jobNum_0/
jobNum_0/FRbatch.py
jobNum_0/params.json
jobNum_0/print_log_54350205.log

jobNum_0/sbatch_config.sh
jobNum_0/batchRun/
jobNum_0/batchRun/configs/

jobNum_0/batchRun/configs/config__batch.yaml
jobNum_0/batchRun/data/
jobNum_0/batchRun/data/session_batch.nwb

jobNum_0/batchRun/viz/

jobNum_0/batchRun/viz/optic_test.avi

jobNum_0/batchRun/viz/demos/
jobNum_0/batchRun/viz/positional/
jobNum_0/batchRun/viz/spectral/
templateFiles/
templateFiles/configTemplate.yaml
templateFiles/sessionOld.nwb
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_17/faceRhythm/.FRbatch.py.gY7Xs8" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG21/2022_05_17/faceRhythm/.dispatcher.py.QCNCsR" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4

copy starting,         time: Sat Jun  4 23:21:19 2022, dir:/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/AEG22/2022_05_13
/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_13/faceRhythm/PE/Rich/analysis_data 

sending incremental file list
created directory /n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_13/faceRhythm

rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_13/faceRhythm/." failed: Operation not permitted (1)

rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_13/faceRhythm/jobNum_0" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_13/faceRhythm/jobNum_0/batchRun" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_13/faceRhythm/jobNum_0/

rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_14/faceRhythm/jobNum_0/.print_log_54350242.log.BTdwg5" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_14/faceRhythm/jobNum_0/.sbatch_config.sh.foDobE" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_14/faceRhythm/jobNum_0/batchRun/configs/.config__batch.yaml.HqEw7c" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_14/faceRhythm/jobNum_0/batchRun/data/.session_batch.nwb.UrzZ4L" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_14/faceRhythm/jobNum_0/batchRun/viz/.optic_test.avi.hekH9p" failed: Operation not permitte

jobNum_0/print_log_54350244.log
jobNum_0/sbatch_config.sh
jobNum_0/batchRun/
jobNum_0/batchRun/configs/
jobNum_0/batchRun/configs/config__batch.yaml
jobNum_0/batchRun/data/
jobNum_0/batchRun/data/session_batch.nwb

jobNum_0/batchRun/viz/

jobNum_0/batchRun/viz/optic_test.avi

jobNum_0/batchRun/viz/demos/
jobNum_0/batchRun/viz/positional/
jobNum_0/batchRun/viz/spectral/
templateFiles/

templateFiles/configTemplate.yaml
templateFiles/sessionOld.nwb
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_17/faceRhythm/.FRbatch.py.qCj5D5" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_17/faceRhythm/.dispatcher.py.XknOrZ" failed: Operation not permitted (1)
rsync: chgrp "/n/files/Neurobio/MICROSCOPE/Rich/analysis_data/face_rhythm_paper/fig_4_2pRAM/AEG22/2022_05_17/faceRhythm/.notes.txt.EJFOgT" failed: Operation not permitted (1)
rsync: chgrp "/n/fi

In [20]:
# ssh_t.close()

In [ ]:
# del ssh_t